<a href="https://colab.research.google.com/github/bloodfield2/ProyectoVision/blob/main/Proyecto_de_visi%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto de Visión 
#     Practical Self Calibration Using a Planar Scene
## Estudiantes
Chin Parra Keny

Chávez Aparicio Edgar Iván

##    Guía

1. cargar los siguientes archivos a /content/:
+ calibrate/
+ rookie.py


#    EN PROGRESO:

Se está probando el código para el caso de un solo par de imágenes para
después extender a N.

#    Instrucciones:

Trabajar las funciones de forma modular en `rookie.py`
Lo que se esté trabajando, desarrollarlo en el block de notas
Una vez terminadas las faces, hacer un módulo si está muy grande

#    TODO

##  ~Para un solo par de imágnes~
+ ~Feature Matching~
+ ~Projective reconstruction~
+ ~Projective Boundle Adjustment~
+ ~Homography based calibration~
+ ~Metric reconstruction~


##  ~Para N imágnes~
+ ~Feature Matching~
+ ~Projective reconstruction~
+ ~Projective Boundle Adjustment~
+ ~Homography based calibration~
+ ~Metric reconstruction~

In [1]:
import numpy as np
import cv2 
import glob
from rookie  import *
from scipy.optimize import minimize
from scipy.optimize import minimize_scalar


In [2]:
images = glob.glob('calibrate/*.jpg') # al menos 15
objpoints, imgpoints, rets =  chesscalib(images)
ret, mtx, dist, rvecs, tvecs = rets
error = err_calib(objpoints, imgpoints,rets)
print( "total error: ",error )
print(mtx)

total error:  0.04249816179320835
[[867.11547226   0.         290.0935642 ]
 [  0.         868.58681435 252.66703266]
 [  0.           0.           1.        ]]


In [3]:
 #   images[0] is the reference
img1 = cv2.imread(images[0],0)

Hlist = []
Referencepoints = []
imagespoints = []

for i in range(1,len(images)):
    
    #   Step 1: Feature Matching
    img2 = cv2.imread(images[i],0)
    points1, points2 = Features(img1,img2)
    Referencepoints.append(points1)
    imagespoints.append(points2)
    
    #   Step 2: First Proyective resconstruction
    h, status = cv2.findHomography(points1, points2, cv2.RANSAC,0.5)
    Hlist.append(h)


In [4]:
# Part 3: Projective Boundle adjustment
x = np.array([1,1,1,1]) # P0, d
center = np.array(list(img1.shape))/2
center = center.reshape((2,1))
args = (Referencepoints,imagespoints,Hlist,center)
#PBA(x,points1,points2,h,np.array(list(img1.shape))/2.0 )
xmin = minimize(PBA, x,args = args, method='nelder-mead',options={'maxiter' :2000,'disp': True},tol = 10**-7)
print(xmin)
xmin = xmin['x']
p0 = xmin[:2]
d = xmin[2:]
print('P0 = ',p0)
print('d = ',d)

Optimization terminated successfully.
         Current function value: 55637.602976
         Iterations: 799
         Function evaluations: 1421
 final_simplex: (array([[-4.53865010e-01,  5.01934196e+00,  5.68568183e-08,
        -3.06490973e-13],
       [-4.53865010e-01,  5.01934196e+00,  5.68580744e-08,
        -3.06492344e-13],
       [-4.53865010e-01,  5.01934196e+00,  5.68569444e-08,
        -3.06490860e-13],
       [-4.53865010e-01,  5.01934196e+00,  5.68572174e-08,
        -3.06492522e-13],
       [-4.53865010e-01,  5.01934196e+00,  5.68574824e-08,
        -3.06492964e-13]]), array([55637.60297586, 55637.60297587, 55637.60297587, 55637.60297587,
       55637.60297591]))
           fun: 55637.60297585678
       message: 'Optimization terminated successfully.'
          nfev: 1421
           nit: 799
        status: 0
       success: True
             x: array([-4.53865010e-01,  5.01934196e+00,  5.68568183e-08, -3.06490973e-13])
P0 =  [-0.45386501  5.01934196]
d =  [ 5.68568183e-08

In [5]:

#   part 4: Homography self calibration
for i in range(len(Hlist)):
    Hlist[i] /= norm(Hlist[i])
args = (Hlist)
xmin = minimize_scalar(residuals, args = args,tol = 10**-7)
print(xmin)
f = xmin['x']
print('f = ',f)

#for i in range(len(Hlist)):
    #Hlist[i] /= norm(Hlist[i])
args = (Hlist)
x = np.array([f,p0[0],f,p0[1],0,0,1])
xmin = minimize(FNF, x,args = args, method='nelder-mead',options={'disp': True},tol = 10**-7)
print(xmin)
xmin = xmin['x']
f = np.array([xmin[0],xmin[2]])
p0 = np.array([xmin[1],xmin[3]]).reshape((2,1))
n0 = xmin[4:]
print('P0 = ',p0)
print('f = ',f)
print('n0 = ',n0)


     fun: 0.0009660505280568198
    nfev: 15
     nit: 11
 success: True
       x: 1.1441351800235494e-11
f =  1.1441351800235494e-11
 final_simplex: (array([[ 1.31036759e-11, -4.36878016e-01,  7.83794374e-12,
         4.31121588e+00,  8.06569090e-04, -3.70844763e-14,
         1.20505009e+00],
       [ 1.30880266e-11, -4.37037933e-01,  7.87186655e-12,
         4.31788224e+00,  8.01329504e-04, -3.71866640e-14,
         1.20311973e+00],
       [ 1.30645657e-11, -4.37277677e-01,  7.92272278e-12,
         4.32787629e+00,  7.93474444e-04, -3.73364717e-14,
         1.20022578e+00],
       [ 1.30799400e-11, -4.37120569e-01,  7.88939601e-12,
         4.32132705e+00,  7.98621972e-04, -3.72393646e-14,
         1.20212222e+00],
       [ 1.30761228e-11, -4.37159576e-01,  7.89767038e-12,
         4.32295310e+00,  7.97343944e-04, -3.72639039e-14,
         1.20165137e+00],
       [ 1.30605470e-11, -4.37318742e-01,  7.93143398e-12,
         4.32958817e+00,  7.92128945e-04, -3.73615703e-14,
         1.

In [6]:
#   part 5: Metric Reconstruction
K = np.array([[xmin[0],0,xmin[1]],[0,xmin[2],xmin[3]],[0,0,1]])
x = np.concatenate([K.reshape((9)),d])    #   K d 
args = (n0,Hlist,p0,Referencepoints,imagespoints)
res = MRF(x,*args)
xmin = minimize(MRF, x,args = args, method='nelder-mead',options={'disp': True},tol = 10**-7)
print(xmin)
xmin = xmin['x']
K = xmin[:9]
K = K.reshape((3,3))
K /= K[2,2]
print(K)
print(mtx)


 final_simplex: (array([[ 2.14690004e-10,  3.08475030e-01,  3.36719647e+00,
        -1.44611468e-01, -2.21481667e-10,  3.16485927e+00,
         1.57264880e-01, -1.88894973e-01,  1.17841875e-02,
        -8.68668133e-08,  5.11333878e-13],
       [ 2.14704427e-10,  3.08511975e-01,  3.36663673e+00,
        -1.44650283e-01, -2.21443737e-10,  3.16343707e+00,
         1.57279949e-01, -1.88937652e-01,  1.17705724e-02,
        -8.70922475e-08,  5.10953037e-13],
       [ 2.14679214e-10,  3.08448011e-01,  3.36737140e+00,
        -1.44588398e-01, -2.21490266e-10,  3.16384755e+00,
         1.57252122e-01, -1.88870008e-01,  1.17897668e-02,
        -8.67589432e-08,  5.11348642e-13],
       [ 2.14700418e-10,  3.08503002e-01,  3.36659084e+00,
        -1.44644671e-01, -2.21439929e-10,  3.16268748e+00,
         1.57275204e-01, -1.88931523e-01,  1.17699771e-02,
        -8.70766030e-08,  5.10894856e-13],
       [ 2.14525817e-10,  3.08188046e-01,  3.36518671e+00,
        -1.44445648e-01, -2.21350074e-10,  3

In [6]:
#   Test
m = points1.shape[0]
test_pref = np.concatenate([points1.T,np.ones((1,m))])
test_pref = h@test_pref 
test_pref = test_pref[:2,:]/test_pref[2,:]
test_pref = Distorted(test_pref,xmin[:2].reshape((2,1)),xmin[2:4])
norms = np.linalg.norm(test_pref-points2,axis = 1)
print(norms.sum(),norms.mean(),norms.max())
test_pref = np.concatenate([points1.T,np.ones((1,m))])
test_pref = h@test_pref 
test_pref = test_pref[:2,:]/test_pref[2,:]
norms = np.linalg.norm(test_pref.T-points2,axis = 1)
print(norms.sum(),norms.mean(),norms.max())

74.27147400871847 0.9902863201162463 5.761299192945103
74.27576287630956 0.9903435050174607 5.767173511835501
